In [1]:
"""
The task given was to create the model in the Model.png, with 2 layers of convolution and pooling, and one layer of dense (fully-connected layer)
I use the nn.py from our lecturer as a base, and works mostly inside the class, within the init and logits-method
"""

import torch
import torch.nn as nn
import torchvision

# Load observations from the mnist dataset. The observations are divided into a training set and a test set
mnist_train = torchvision.datasets.MNIST('./data', train=True, download=True)
# torch.functional.nn.conv2d argument must include channels (1)
x_train = mnist_train.data.reshape(-1, 1, 28, 28).float()
# Create output tensor
y_train = torch.zeros((mnist_train.targets.shape[0], 10))
y_train[torch.arange(mnist_train.targets.shape[0]),
        mnist_train.targets] = 1  # Populate output

mnist_test = torchvision.datasets.MNIST('./data', train=False, download=True)
# torch.functional.nn.conv2d argument must include channels (1)
x_test = mnist_test.data.reshape(-1, 1, 28, 28).float()
y_test = torch.zeros((mnist_test.targets.shape[0], 10))  # Create output tensor
y_test[torch.arange(mnist_test.targets.shape[0]),
       mnist_test.targets] = 1  # Populate output

# Normalization of inputs
mean = x_train.mean()
std = x_train.std()
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Divide training data into batches to speed up optimization
batches = 600
x_train_batches = torch.split(x_train, batches)
y_train_batches = torch.split(y_train, batches)


class ConvolutionalNeuralNetworkModel(nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNetworkModel, self).__init__()

        # Initialises the Model layers (includes initialized model variables):
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv_first = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.conv_second = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.dense = nn.Linear(64 * 7 * 7, 10)

    def logits(self, x):
        # Here is the model layers in the right order
        x = self.pool(self.conv_first(x)) # First convolutional layer and pooling
        x = self.pool(self.conv_second(x)) # Second convolutional layer and pooling
        return self.dense(x.view(-1, 64 * 7 * 7))

    # Predictor
    def f(self, x):
        return torch.softmax(self.logits(x), dim=1)

    # Cross Entropy loss
    def loss(self, x, y):
        return nn.functional.cross_entropy(self.logits(x), y.argmax(1))

    # Accuracy
    def accuracy(self, x, y):
        return torch.mean(torch.eq(self.f(x).argmax(1), y.argmax(1)).float())


model = ConvolutionalNeuralNetworkModel()

epoch = 20
learning_rate = 0.001

# Optimize: adjust W and b to minimize loss using stochastic gradient descent
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for i, epoch in enumerate(range(epoch)):
    for batch in range(len(x_train_batches)):
        # Compute loss gradients
        model.loss(x_train_batches[batch], y_train_batches[batch]).backward()
        optimizer.step()  # Perform optimization by adjusting W and b,
        optimizer.zero_grad()  # Clear gradients for next step

    print(f'Accuracy in epoch {i+1}: {model.accuracy(x_test, y_test)*100}%')

Accuracy in epoch 1: 97.05999755859375%
Accuracy in epoch 2: 98.11000061035156%
Accuracy in epoch 3: 98.32999420166016%
Accuracy in epoch 4: 98.43999481201172%
Accuracy in epoch 5: 98.6199951171875%
Accuracy in epoch 6: 98.63999938964844%
Accuracy in epoch 7: 98.22000122070312%
Accuracy in epoch 8: 98.30999755859375%
Accuracy in epoch 9: 98.36000061035156%
Accuracy in epoch 10: 98.40999603271484%
Accuracy in epoch 11: 98.38999938964844%
Accuracy in epoch 12: 98.73999786376953%
Accuracy in epoch 13: 98.69999694824219%
Accuracy in epoch 14: 98.54999542236328%
Accuracy in epoch 15: 98.58999633789062%
Accuracy in epoch 16: 98.23999786376953%
Accuracy in epoch 17: 98.72999572753906%
Accuracy in epoch 18: 98.54999542236328%
Accuracy in epoch 19: 98.43999481201172%
Accuracy in epoch 20: 98.65999603271484%


In [2]:
"""
The task given was to create the model in the Model.png, with 2 layers of convolution and pooling, and two layers of dense (fully-connected layer)
I use the nn.py from our lecturer as a base, and works mostly inside the class, within the init and logits-method
"""

import torch
import torch.nn as nn
import torchvision

# Load observations from the mnist dataset. The observations are divided into a training set and a test set
mnist_train = torchvision.datasets.MNIST('./data', train=True, download=True)
# torch.functional.nn.conv2d argument must include channels (1)
x_train = mnist_train.data.reshape(-1, 1, 28, 28).float()
# Create output tensor
y_train = torch.zeros((mnist_train.targets.shape[0], 10))
y_train[torch.arange(mnist_train.targets.shape[0]),
        mnist_train.targets] = 1  # Populate output

mnist_test = torchvision.datasets.MNIST('./data', train=False, download=True)
# torch.functional.nn.conv2d argument must include channels (1)
x_test = mnist_test.data.reshape(-1, 1, 28, 28).float()
y_test = torch.zeros((mnist_test.targets.shape[0], 10))  # Create output tensor
y_test[torch.arange(mnist_test.targets.shape[0]),
       mnist_test.targets] = 1  # Populate output

# Normalization of inputs
mean = x_train.mean()
std = x_train.std()
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Divide training data into batches to speed up optimization
batches = 600
x_train_batches = torch.split(x_train, batches)
y_train_batches = torch.split(y_train, batches)

"""
Except for the printing, the only place we change the model is in the logitsmethod, and define new methods in init
"""
class ConvolutionalNeuralNetworkModel(nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNetworkModel, self).__init__()

        # Initialises the Model layers (includes initialized model variables):
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv_first = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.conv_second = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.dense_first = nn.Linear(64 * 7 * 7, 1024) # Other sources call these fc1 and fc2
        self.dense_second = nn.Linear(1024, 10)

    def logits(self, x):
        # Here is the model layers in the right order
        x = self.pool(self.conv_first(x)) # First convolutional layer and pooling
        x = self.pool(self.conv_second(x)) # Second convolutional layer and pooling
        x = self.dense_first(x.view(-1, 64 * 7 * 7))
        x = self.dense_second(x.view(-1, 1024)) # Two dense layers
        return x

    # Predictor
    def f(self, x):
        return torch.softmax(self.logits(x), dim=1)

    # Cross Entropy loss
    def loss(self, x, y):
        return nn.functional.cross_entropy(self.logits(x), y.argmax(1))

    # Accuracy
    def accuracy(self, x, y):
        return torch.mean(torch.eq(self.f(x).argmax(1), y.argmax(1)).float())


model = ConvolutionalNeuralNetworkModel()

epoch = 20
learning_rate = 0.001

# Optimize: adjust W and b to minimize loss using stochastic gradient descent
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for i, epoch in enumerate(range(epoch)):
    for batch in range(len(x_train_batches)):
        # Compute loss gradients
        model.loss(x_train_batches[batch], y_train_batches[batch]).backward()
        optimizer.step()  # Perform optimization by adjusting W and b,
        optimizer.zero_grad()  # Clear gradients for next step

    print(f'Accuracy in epoch {i+1}: {model.accuracy(x_test, y_test)*100}%')


Accuracy in epoch 1: 97.65999603271484%
Accuracy in epoch 2: 98.11000061035156%
Accuracy in epoch 3: 98.3699951171875%
Accuracy in epoch 4: 98.12999725341797%
Accuracy in epoch 5: 98.07999420166016%
Accuracy in epoch 6: 97.8499984741211%
Accuracy in epoch 7: 98.08999633789062%
Accuracy in epoch 8: 97.88999938964844%
Accuracy in epoch 9: 97.93000030517578%
Accuracy in epoch 10: 97.89999389648438%
Accuracy in epoch 11: 97.43000030517578%
Accuracy in epoch 12: 98.1500015258789%
Accuracy in epoch 13: 98.13999938964844%
Accuracy in epoch 14: 97.68999481201172%
Accuracy in epoch 15: 98.30999755859375%
Accuracy in epoch 16: 98.08999633789062%
Accuracy in epoch 17: 97.7199935913086%
Accuracy in epoch 18: 97.7699966430664%
Accuracy in epoch 19: 98.05999755859375%
Accuracy in epoch 20: 98.22000122070312%


In [13]:
"""
The task given was to improve the model using for example ReLU or dropout
I use the nn.py from our lecturer as a base, and works mostly inside the class, within the init and logits-method
"""
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

# Load observations from the mnist dataset. The observations are divided into a training set and a test set
mnist_train = torchvision.datasets.MNIST(
    './data', train=True, download=True)
# torch.functional.nn.conv2d argument must include channels (1)
x_train = mnist_train.data.reshape(-1, 1, 28, 28).float()
# Create output tensor
y_train = torch.zeros((mnist_train.targets.shape[0], 10))
y_train[torch.arange(mnist_train.targets.shape[0]),
        mnist_train.targets] = 1  # Populate output

mnist_test = torchvision.datasets.MNIST(
    './data', train=False, download=True)
# torch.functional.nn.conv2d argument must include channels (1)
x_test = mnist_test.data.reshape(-1, 1, 28, 28).float()
y_test = torch.zeros((mnist_test.targets.shape[0], 10))  # Create output tensor
y_test[torch.arange(mnist_test.targets.shape[0]),
       mnist_test.targets] = 1  # Populate output

# Normalization of inputs
mean = x_train.mean()
std = x_train.std()
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Divide training data into batches to speed up optimization
batches = 600
x_train_batches = torch.split(x_train, batches)
y_train_batches = torch.split(y_train, batches)


class ConvolutionalNeuralNetworkModel(nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNetworkModel, self).__init__()
        # Initialises the Model layers (includes initialized model variables):
        # Could also write stride=2 here, but I suspect it is the standard for kernelsize 2 (as 2x2 will jump 2 to the right for the next 2x2)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(64*7*7, 1024)
        self.fc2 = nn.Linear(1024, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        # x = self.pool(self.conv1(x))
        # x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.dropout(F.relu(self.conv1(x)), p=0.1))
        
        # x = self.pool(self.conv2(x))
        # x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.dropout(F.relu(self.conv2(x)), p=0.1))
        
        x = self.fc1(x.view(-1, 64 * 7 * 7))
        x = self.fc2(x.view(-1, 1024))
        x = self.fc3(x.view(-1, 100))
        return x

    # Predictor
    def f(self, x):
        return torch.softmax(self.forward(x), dim=1)

    # Cross Entropy loss
    def loss(self, x, y):
        return nn.functional.cross_entropy(self.forward(x), y.argmax(1))

    # Accuracy
    def accuracy(self, x, y):
        return torch.mean(torch.eq(self.f(x).argmax(1), y.argmax(1)).float())


model = ConvolutionalNeuralNetworkModel()

epoch = 20
learning_rate = 0.001

# Optimize: adjust W and b to minimize loss using stochastic gradient descent
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for i, epoch in enumerate(range(epoch)):
    for batch in range(len(x_train_batches)):
        # Compute loss gradients
        model.loss(x_train_batches[batch], y_train_batches[batch]).backward()
        optimizer.step()  # Perform optimization by adjusting W and b,
        optimizer.zero_grad()  # Clear gradients for next step

    print(f'Accuracy in epoch {i+1}: {model.accuracy(x_test, y_test)*100}%')


Accuracy in epoch 1: 98.11000061035156%
Accuracy in epoch 2: 98.48999786376953%
Accuracy in epoch 3: 98.76000213623047%
Accuracy in epoch 4: 98.5%
Accuracy in epoch 5: 98.5199966430664%
Accuracy in epoch 6: 98.7699966430664%
Accuracy in epoch 7: 98.91999816894531%
Accuracy in epoch 8: 98.69999694824219%
Accuracy in epoch 9: 98.43000030517578%
Accuracy in epoch 10: 98.86000061035156%
Accuracy in epoch 11: 98.22000122070312%
Accuracy in epoch 12: 99.08000183105469%
Accuracy in epoch 13: 98.41999816894531%
Accuracy in epoch 14: 98.6199951171875%
Accuracy in epoch 15: 98.7699966430664%
Accuracy in epoch 16: 98.77999877929688%
Accuracy in epoch 17: 98.52999877929688%
Accuracy in epoch 18: 98.97999572753906%
Accuracy in epoch 19: 98.8699951171875%
Accuracy in epoch 20: 98.91999816894531%


In [5]:
"""
The task given was to try another dataset called Fashion MNIST, and create the best possible accuracy
I use the nn.py from our lecturer as a base, and works mostly inside the class, within the init and forward-method
"""

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

# Load observations from the mnist dataset. The observations are divided into a training set and a test set
mnist_train = torchvision.datasets.FashionMNIST(
    './data', train=True, download=True)
# torch.functional.nn.conv2d argument must include channels (1)
x_train = mnist_train.data.reshape(-1, 1, 28, 28).float()
# Create output tensor
y_train = torch.zeros((mnist_train.targets.shape[0], 10))
y_train[torch.arange(mnist_train.targets.shape[0]),
        mnist_train.targets] = 1  # Populate output

mnist_test = torchvision.datasets.FashionMNIST(
    './data', train=False, download=True)
# torch.functional.nn.conv2d argument must include channels (1)
x_test = mnist_test.data.reshape(-1, 1, 28, 28).float()
y_test = torch.zeros((mnist_test.targets.shape[0], 10))  # Create output tensor
y_test[torch.arange(mnist_test.targets.shape[0]),
       mnist_test.targets] = 1  # Populate output

# Normalization of inputs
mean = x_train.mean()
std = x_train.std()
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Divide training data into batches to speed up optimization
batches = 600
x_train_batches = torch.split(x_train, batches)
y_train_batches = torch.split(y_train, batches)


class ConvolutionalNeuralNetworkModel(nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNetworkModel, self).__init__()
        # Initialises the Model layers (includes initialized model variables):
        # Could also write stride=2 here, but I suspect it is the standard for kernelsize 2 (as 2x2 will jump 2 to the right for the next 2x2)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(64*7*7, 1024)
        self.fc2 = nn.Linear(1024, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        # x = self.pool(self.conv1(x))
        # x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.dropout(F.relu(self.conv1(x)), p=0.1))
        
        # x = self.pool(self.conv2(x))
        # x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.dropout(F.relu(self.conv2(x)), p=0.1))
        
        x = self.fc1(x.view(-1, 64 * 7 * 7))
        x = self.fc2(x.view(-1, 1024))
        x = self.fc3(x.view(-1, 100))
        return x

    # Predictor
    def f(self, x):
        return torch.softmax(self.forward(x), dim=1)

    # Cross Entropy loss
    def loss(self, x, y):
        return nn.functional.cross_entropy(self.forward(x), y.argmax(1))

    # Accuracy
    def accuracy(self, x, y):
        return torch.mean(torch.eq(self.f(x).argmax(1), y.argmax(1)).float())


model = ConvolutionalNeuralNetworkModel()

epoch = 20
learning_rate = 0.001

# Optimize: adjust W and b to minimize loss using stochastic gradient descent
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for i, epoch in enumerate(range(epoch)):
    for batch in range(len(x_train_batches)):
        # Compute loss gradients
        model.loss(x_train_batches[batch], y_train_batches[batch]).backward()
        optimizer.step()  # Perform optimization by adjusting W and b,
        optimizer.zero_grad()  # Clear gradients for next step

    print(f'Accuracy in epoch {i+1}: {model.accuracy(x_test, y_test)*100}%')


Accuracy in epoch 1: 86.0999984741211%
Accuracy in epoch 2: 88.11000061035156%
Accuracy in epoch 3: 88.84000396728516%
Accuracy in epoch 4: 89.56000518798828%
Accuracy in epoch 5: 89.2300033569336%
Accuracy in epoch 6: 89.22000122070312%
Accuracy in epoch 7: 89.44000244140625%
Accuracy in epoch 8: 89.74000549316406%
Accuracy in epoch 9: 90.09000396728516%
Accuracy in epoch 10: 90.29000091552734%
Accuracy in epoch 11: 90.72000122070312%
Accuracy in epoch 12: 90.66000366210938%
Accuracy in epoch 13: 90.97999572753906%
Accuracy in epoch 14: 90.54000091552734%
Accuracy in epoch 15: 90.69999694824219%
Accuracy in epoch 16: 90.4800033569336%
Accuracy in epoch 17: 90.44999694824219%
Accuracy in epoch 18: 90.80999755859375%
Accuracy in epoch 19: 90.81999969482422%
Accuracy in epoch 20: 90.69999694824219%
